In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
 
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage

In [26]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [27]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 23


In [89]:
docs

[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="\n출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책\n\n\n[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은 받는 

In [ ]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 43


In [29]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = UpstageEmbeddings(model="embedding-query")

In [30]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [31]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [32]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("삼성전자가 자체 개발한 AI 의 이름은?")

[Document(id='ca7a5b84-b2a9-48b1-b136-d3db8bde25cf', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': 'SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 12}, page_content='SPRi AI Brief |  \n2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 

In [59]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

In [94]:
# ChatUpstage 객체를 생성합니다.
llm = ChatUpstage(model="solar-mini")

In [93]:
llm.invoke("삼성전자가 자체 개발한 AI의 이름은?")

AIMessage(content='삼성전자가 자체 개발한 AI의 이름은 "삼성 빌드(Samsung Neural) AI"입니다. 이 AI는 다양한 삼성전자 제품에 적용되어 사용자들에게 더 나은 서비스를 제공하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 20, 'total_tokens': 62, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'solar-pro-241126', 'system_fingerprint': None, 'id': '34d1702d-eea8-4dfb-b1b0-07579df6c7d8', 'finish_reason': 'stop', 'logprobs': None}, id='run-02f080b4-2c8b-4ed2-8255-dff205c12bef-0', usage_metadata={'input_tokens': 20, 'output_tokens': 42, 'total_tokens': 62, 'input_token_details': {}, 'output_token_details': {}})

In [65]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [91]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "삼성전자가 자체 개발한 AI 의 이름은?"
response = chain.invoke(question)
print(response)

삼성전자가 자체 개발한 AI의 이름은 '삼성 가우스'입니다.


In [53]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [54]:
prompt = ChatPromptTemplate.from_template("다음의 주제에 대해서 설명해줘: {topic}")
model = ChatUpstage()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

result = chain.invoke({"topic": "LangChain"})
print(result)

LangChain은 AI 및 자연어 처리 분야에서 주목받는 도구로, AI 엔지니어와 개발자들이 대규모 언어 모델(Large Language Models, LLMs)과 파이썬 코드를 효율적으로 통합하고 관리할 수 있도록 도와줍니다. 이 도구는 특히 AI 솔루션의 빌드, 배포 및 유지 관리에 중점을 두고 있습니다.

주요 특징과 이점은 다음과 같습니다:

1. **LLM 통합**: LangChain은 Open Assistant, Llama 2, Mistral, Falcon과 같은 인기 있는 대규모 언어 모델을 지원하여 다양한 LLM을 프로젝트에 쉽게 통합할 수 있습니다.

2. **체인**: 이 도구의 핵심은 "체인"이라는 개념입니다. 체인은 LLM과 다른 기능을 결합하여 복잡한 작업을 처리하는 일련의 단계입니다. 이는 LLM에 대한 여러 호출을 결합하여 더 강력하고 유용한 애플리케이션을 만들 수 있음을 의미합니다.

3. **모듈성**: LangChain은 모듈식 설계로 되어 있어, 사용자는 자체 "플러그인"을 만들어 체인에 통합할 수 있습니다. 이를 통해 사용자는 기존 기능을 확장하거나 완전히 새로운 기능을 추가할 수 있습니다.

4. **효율성**: LangChain은 LLM과 상호 작용하는 동안 효율성을 강조하여 복잡한 작업도 비교적 빠르게 실행할 수 있습니다.

5. **사용자 친화적**: 파이썬을 기반으로 하는 LangChain은 개발자 친화적이며, 챗봇, 문서 분석기 등 다양한 애플리케이션을 쉽게 만들 수 있습니다.

6. **커뮤니티 지원**: 강력한 개발자 커뮤니티를 보유하고 있어, 사용자는 LangChain을 사용하여 만든 프로젝트, 팁 및 트릭을 공유하고 논의할 수 있습니다.

LangChain은 AI 및 NLP 기능을 갖춘 애플리케이션을 만들고자 하는 개발자나 AI 전문가에게 유용한 도구입니다. 이는 대규모 언어 모델을 활용하여 더 정교하고 다재다능한 AI 애플리케이션을 만들 수 있도록 도와줍니다.


In [18]:
prompt = ChatPromptTemplate.from_template("다음의 주제에 대해서 설명해줘: {topic}")
model = ChatUpstage()
output_parser = StrOutputParser()

chain = prompt | model

result = chain.invoke({"topic": "LangChain"})
print(result)

content='LangChain은 대규모 언어 모델을 위한 Python 라이브러리로, 이러한 모델과 상호 작용하고 활용할 수 있는 도구를 제공합니다. 이 라이브러리는 생성형 AI 애플리케이션을 구축하고 배포하기 위한 기능을 제공하는 것을 목표로 합니다.\n\nLangChain의 주요 기능과 구성 요소는 다음과 같습니다:\n\n1. 체인(Chain): LangChain에서 체인은 일련의 단계로 구성된 작업을 수행하는 핵심입니다. 각 단계는 입력에 대해 작동하고 출력을 생성하여 다음 단계의 입력으로 전달되는 함수입니다.\n\n2. 문서 데이터베이스(DocumentDB): 문서 데이터베이스는 챗봇이 검색할 수 있는 정보를 저장하고 관리하는 데 사용됩니다. LangChain은 SQL 및 NoSQL 데이터베이스, Apache Cassandra 및 MongoDB와 같은 인기 있는 데이터베이스와 통합할 수 있습니다.\n\n3. 벡터 저장소(Vector Store): 이는 자연어 처리(NLP)를 위해 최적화된 문서 데이터베이스의 한 유형입니다. 벡터 저장소는 벡터 검색을 지원하여 대규모 텍스트 컬렉션에서 관련 정보를 빠르게 찾을 수 있습니다.\n\n4. 챗봇 에이전트: LangChain은 대규모 언어 모델(LLM)과 상호 작용하기 위한 사용자 지정 가능한 챗봇 에이전트를 제공합니다. 이러한 에이전트는 사용자와 LLM 간의 중개자 역할을 하며, 챗봇에 특정 기능을 추가할 수 있습니다.\n\n5. 도구(Tools): LangChain은 다양한 도구를 지원하여 챗봇의 기능을 확장합니다. 이러한 도구는 웹 검색, PDF 변환 또는 복잡한 계산과 같은 작업을 수행할 수 있습니다.\n\n6. 커스텀화: 개발자는 LangChain의 기능을 확장하고 특정 챗봇 요구 사항을 충족하기 위해 자체 체인, 문서 데이터베이스, 벡터 저장소, LLM, 도구 및 챗봇 에이전트를 만들 수 있습니다.\n\nLangChain은 개발자가 사용자 지정 가능한 LLM 기반 애플리케이션을 구축하고 배포할 수 있

# 네이버 뉴스기사

In [67]:
from dotenv import load_dotenv
load_dotenv()

True

In [69]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [70]:
bs4.SoupStrainer(
    "div",
    attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
)

<SoupStrainer name=[<TagNameMatchRule string=div pattern=None function=None present=None>] attrs=defaultdict(<class 'list'>, {'class': [<AttributeValueMatchRule string=newsct_article _article_body pattern=None function=None present=None>, <AttributeValueMatchRule string=media_end_head_title pattern=None function=None present=None>]}) string=[]>

In [71]:
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs


문서의 수: 1


[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="\n출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책\n\n\n[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은 받는 

In [72]:
ext_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

3

In [73]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [75]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

In [77]:
# ChatUpstage 객체를 생성합니다.
llm = ChatUpstage(model="solar-mini")

# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [82]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "부영그룹의 출산 장려 정책에 대해 설명해주세요."
response = rag_chain.invoke(question)
print(response)

부영그룹은 저출생 대책으로 2021년 이후 태어난 직원 자녀에게 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다. 해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다. 만일 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.


In [ ]:
from langchain_teddynote.messages import stream_response

answer = rag_chain.stream("부영그룹의 출산 장려 정책에 대해 설명해주세요.")
stream_response(answer)

부영그룹은 2021년 이후 태어난 직원 자녀에게 1억원씩, 총 70억원을 지원하고, 연년생과 쌍둥이 자녀가 있는 경우 총 2억원을 지원하는 출산 장려 정책을 시행하고 있습니다. 또한, 셋째를 낳는 경우 국민주택을 제공하겠다는 계획도 밝혔습니다.

In [85]:
answer = rag_chain.stream("부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?")
stream_response(answer)

부영그룹은 2021년 이후 태어난 직원 자녀에게 1억원씩 지원하고 있습니다.

In [87]:
answer = rag_chain.stream("정부의 저출생 대책을 bullet points 형식으로 작성해 주세요.")
stream_response(answer)

* 부모 급여: 0세 아이에게 매달 100만원 지급
* 첫만남이용권: 아이 출생 시 1회 지급
* 아동수당: 만 8세 미만 아동에게 매월 지급
* 인천시 지원금: 새로 태어난 아기에게 18세까지 1억원 지급
* 광주시 지원금: 17세까지 7400만원 지급
* 회사 지원금: 일부 회사에서 출산한 직원에게 1억원 지급
* 세금 혜택: 출산장려금에 대한 면세 제안

In [88]:
answer = rag_chain.stream("부영그룹의 임직원 숫자는 몇명인가요?")
stream_response(answer)

주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.